In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time

from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)

In [4]:
cities=[]

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
        
len(cities)

593

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
city_data = []

print("Beginning Data Retrieval     ")
print("-----------------------------")

record_count = 1
set_count = 1

for i, city in enumerate(cities):

    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    city_url = url + "&q=" + city.replace(" ","+")

    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    record_count += 1
    
    try:
        city_weather = requests.get(city_url).json()
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    except:
        print("City not found. Skipping...")
        pass

print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | chara
Processing Record 2 of Set 1 | svetlaya
Processing Record 3 of Set 1 | cacapava do sul
Processing Record 4 of Set 1 | makakilo city
Processing Record 5 of Set 1 | maputo
Processing Record 6 of Set 1 | andra
Processing Record 7 of Set 1 | rikitea
Processing Record 8 of Set 1 | shilka
Processing Record 9 of Set 1 | labuan
Processing Record 10 of Set 1 | katangli
Processing Record 11 of Set 1 | daru
Processing Record 12 of Set 1 | kapaa
Processing Record 13 of Set 1 | coihaique
Processing Record 14 of Set 1 | waipawa
Processing Record 15 of Set 1 | port-cartier
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | mar del plata
Processing Record 18 of Set 1 | beringovskiy
Processing Record 19 of Set 1 | dongsheng
Processing Record 20 of Set 1 | pampa
Processing Record 21 of Set 1 | lakatoro
Processing Record 22 of Set 1 | bonfim
Processing Record 23 of Set 1 | tuy hoa
Process

Processing Record 37 of Set 4 | kahului
Processing Record 38 of Set 4 | ipixuna
Processing Record 39 of Set 4 | inhambane
Processing Record 40 of Set 4 | zelenikovo
Processing Record 41 of Set 4 | termoli
Processing Record 42 of Set 4 | dongning
Processing Record 43 of Set 4 | ulaanbaatar
Processing Record 44 of Set 4 | annau
Processing Record 45 of Set 4 | tasiilaq
Processing Record 46 of Set 4 | kenai
Processing Record 47 of Set 4 | tabialan
City not found. Skipping...
Processing Record 48 of Set 4 | portland
Processing Record 49 of Set 4 | aguimes
Processing Record 50 of Set 4 | cidreira
Processing Record 1 of Set 5 | russell
Processing Record 2 of Set 5 | kudahuvadhoo
Processing Record 3 of Set 5 | chifeng
Processing Record 4 of Set 5 | riyadh
Processing Record 5 of Set 5 | karratha
Processing Record 6 of Set 5 | champerico
Processing Record 7 of Set 5 | mount isa
Processing Record 8 of Set 5 | deputatskiy
Processing Record 9 of Set 5 | vaitupu
City not found. Skipping...
Processin

Processing Record 24 of Set 8 | sur
Processing Record 25 of Set 8 | hastings
Processing Record 26 of Set 8 | bonthe
Processing Record 27 of Set 8 | balti
Processing Record 28 of Set 8 | baillif
Processing Record 29 of Set 8 | aqtobe
Processing Record 30 of Set 8 | valdivia
Processing Record 31 of Set 8 | havre-saint-pierre
Processing Record 32 of Set 8 | trinidad
Processing Record 33 of Set 8 | khonuu
City not found. Skipping...
Processing Record 34 of Set 8 | puerto carreno
Processing Record 35 of Set 8 | necochea
Processing Record 36 of Set 8 | laela
Processing Record 37 of Set 8 | guiyang
Processing Record 38 of Set 8 | acarau
Processing Record 39 of Set 8 | rio grande
Processing Record 40 of Set 8 | kuznechnoye
Processing Record 41 of Set 8 | mackay
Processing Record 42 of Set 8 | roebourne
Processing Record 43 of Set 8 | arman
Processing Record 44 of Set 8 | morehead
Processing Record 45 of Set 8 | shingu
Processing Record 46 of Set 8 | road town
Processing Record 47 of Set 8 | lo

Processing Record 14 of Set 12 | gudalur
Processing Record 15 of Set 12 | satitoa
City not found. Skipping...
Processing Record 16 of Set 12 | tuggurt
City not found. Skipping...
Processing Record 17 of Set 12 | dzhusaly
City not found. Skipping...
Processing Record 18 of Set 12 | makinsk
Processing Record 19 of Set 12 | buraydah
Processing Record 20 of Set 12 | smiltene
Processing Record 21 of Set 12 | san quintin
Processing Record 22 of Set 12 | suluq
Processing Record 23 of Set 12 | hwange
Processing Record 24 of Set 12 | sabha
Processing Record 25 of Set 12 | waterloo
Processing Record 26 of Set 12 | jawhar
Processing Record 27 of Set 12 | bac lieu
Processing Record 28 of Set 12 | tual
Processing Record 29 of Set 12 | artyk
City not found. Skipping...
Processing Record 30 of Set 12 | ashibetsu
Processing Record 31 of Set 12 | matagami
Processing Record 32 of Set 12 | drayton valley
Processing Record 33 of Set 12 | ciudad bolivar
Processing Record 34 of Set 12 | baghmara
Processing 

In [10]:
city_data_df=pd.DataFrame(city_data)
new_column_order = ['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']
city_data_df=city_data_df[new_column_order]

output_data_file = "cities.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [9]:
pwd

'C:\\Users\\Jack Bauer\\Desktop\\Class Repository\\Module 6- APIs\\world_weather_analysis'